In [1]:
using DataFrames
using ControlSystemsBase
using Statistics

using RealTimeScheduling
using ControlTimingSafety

push!(LOAD_PATH, "../lib")
using Experiments
using Benchmarks

[ Info: Precompiling ControlTimingSafety [3a76c758-31f1-42cb-b697-12fe755e3b12]
[ Info: Precompiling Experiments [top-level]
[ Info: Precompiling Benchmarks [top-level]


## SHT

Address the issue about equal prior belief for $H_0$ and $H_1$. Recall that given deviation upper bound $d_{ub}$, confidence level $c$, and the set of all possible samples $X$, the two hypothesis are defined as follows:
$$
\DeclareMathOperator{\dev}{Deviation}
H_0: Pr(\dev(x) < d_{ub} | x \in X) < c \\
H_1: Pr(\dev(x) < d_{ub} | x \in X) > c
$$
assuming that the sample $x$ is drawn uniformly from the set $X$.

In the paper, the Bayes Factor $B$ is used to compute $K$, the number of required samples to draw to give 
\begin{align}
\frac{Pr(H_1)}{Pr(H_0)} B &= \frac{Pr(H_1|Data)}{Pr(H_0|Data)} \\
\frac{Pr(H_1)}{Pr(H_0)} B &= \frac{\frac{Pr(Data|H_1)Pr(H_1)}{Pr(Data)}}{\frac{Pr(Data|H_0)Pr(H_0)}{Pr(Data)}} \\
B &= \frac{Pr(Data|H_1)}{Pr(Data|H_0)} \\
B &= \frac{\int_c^1 P(Data|\theta=q, H_1) f_{\theta|H_1}(q)dq}
          {\int_0^c P(Data|\theta=q, H_0) f_{\theta|H_0}(q)dq} \\
B &= \frac{\int_c^1 P(Data|\theta=q, H_1) \frac{1}{1-c} dq}
          {\int_0^c P(Data|\theta=q, H_0) \frac{1}{c} dq} \\
B &= \frac{c \int_c^1 q^K dq}
          {(1-c) \int_0^c q^K dq} \\
\frac{1-c}{c} B &= \frac{\int_c^1 q^K dq}
                        {\int_0^c q^K dq} \\
K &= -\log_c(\frac{1-c}{c}B + 1)
\end{align}

<!-- Traditionally, a Bayes factor $B$ of $100$ is considered extreme evidence favoring the alternative hypothesis. However, the paper we are based on states the type-I error is expressed by
$$
err = \frac{c}{c(1-c)B}
$$
Which is ~$.497$ when $B=100$, and ~$.000238$ when $B=4.15e5$. -->

In [25]:
let
    c = 0.99
    B = 100
    e = c/(c+(1-c)*B)
    B_og = 4.15e5
    e_og = c/(c+(1-c)*B_og)
    [e e_og]
end

1×2 Matrix{Float64}:
 0.497487  0.000238497

In [38]:
let
    c = [0.9, 0.99, 0.999, 0.9999, 1-1e-10, 1-1e-15, 1-1e-16]
    [-log.(c, c./ (1 .- c)) -log.(c, 4.15e5+1)]
end

7×2 Matrix{Float64}:
    20.8543        122.779
   457.211        1287.12
  6903.3         12929.6
 92097.8             1.29354e5
     2.30258e11      1.2936e11
     3.45672e16      1.29464e16
     3.30896e17      1.16517e17

In [42]:
let
    B = 4.15e5
    c = [0.9, 0.99, 0.999, 0.9999, 1-1e-10, 1-1e-15, 1-1e-16]
    # effective_B = c ./ (1 .- c) * B
    effective_B = (1 .- c) ./ c * B
    K = -log.(c, effective_B .+ 1)
    K_og = -log.(c, B+1)

    round.([c K_og K], sigdigits=4)
end

7×3 Matrix{Float64}:
 0.9        122.8          101.9
 0.99      1287.0          829.9
 0.999    12930.0         6029.0
 0.9999  129400.0        37490.0
 1.0          1.294e11  415000.0
 1.0          1.295e16  415000.0
 1.0          1.165e17  415000.0

In [43]:
let
    B = 100
    c = [0.9, 0.99, 0.999, 0.9999, 1-1e-10, 1-1e-15, 1-1e-16]
    # effective_B = c ./ (1 .- c) * B
    effective_B = (1 .- c) ./ c * B
    K = -log.(c, effective_B .+ 1)
    K_og = -log.(c, B+1)

    round.([c K_og K], sigdigits=4)
end

7×3 Matrix{Float64}:
 0.9        43.8        23.67
 0.99      459.2        69.47
 0.999    4613.0        95.35
 0.9999  46150.0        99.51
 1.0         4.615e10  100.0
 1.0         4.619e15  100.0
 1.0         4.157e16  100.0

## Experiments

### pWCET Sampler

In [11]:
sp = SamplerPWCET(0.9, 100)

SamplerPWCET(0.9, 100)

In [17]:
mean([sum(rand(sp)) for i in 1:100])

90.31

### System Dynamics

Open-loop stable system:
$$
\mathbf{\dot{x}}(t) = \begin{bmatrix} -6.0 & 1.0 \\ 0.2 & -0.7 \end{bmatrix} \mathbf{x}(t) +
\begin{bmatrix} 5.0 \\ 0.5 \end{bmatrix} u(t)
$$
where
$$
\mathbf{x}(t) = \begin{bmatrix} x_1(t) \\ x_2(t) \end{bmatrix}
$$
is the system state and $u(t)$ is the control input.

We also have a open-loop unstable system:
$$
\mathbf{\dot{x}}(t) = \begin{bmatrix} 0 & 6.5 \\ 0 & 0 \end{bmatrix} \mathbf{x}(t) +
\begin{bmatrix} 0 \\ 19.685 \end{bmatrix} u(t)
$$

### Deviation Estimation

In [31]:
# Setting parameters for the experiment
H = 100
c = 0.99
B = 4.15e5

1745

In [34]:
sys = benchmarks[:RCN]
h = 0.02

# Construct an automaton with no constraint
a = hold_kill(c2d(sys, h), delay_lqr(sys, h))
estimate_deviation(a, sp, 

StateSpace{Continuous, Float64}
A = 
 -6.0   1.0
  0.2  -0.7
B = 
 5.0
 0.5
C = 
 1.0  0.0
 0.0  1.0
D = 
 0.0
 0.0

Continuous-time state-space model